In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
sns.set_style("white")

In [ ]:
preds_0612 = np.array(pd.read_csv('report/inference_06-12.csv').iloc[:, 1])
preds_0102 = np.array(pd.read_csv('report/inference_01-02.csv').iloc[:, 1])
preds_0305 = np.array(pd.read_csv('report/inference_03-05.csv').iloc[:, 1])
preds_0305_v2 = np.array(pd.read_csv('report/inference_03-05_v2.csv').iloc[:, 1])

preds_0612 = np.array(pd.read_csv('report/inference_06-12_reranked.csv').iloc[:, 1])
preds_0102 = np.array(pd.read_csv('report/inference_01-02_reranked.csv').iloc[:, 1])
preds_0305 = np.array(pd.read_csv('report/inference_03-05_reranked.csv').iloc[:, 1])
preds_0305_v2 = np.array(pd.read_csv('report/inference_03-05_v2_reranked.csv').iloc[:, 1])

In [ ]:
plot = sns.kdeplot(preds_0612, shade=False)
plot = sns.kdeplot(preds_0102, shade=False)
plot = sns.kdeplot(preds_0305, shade=False)
plot = sns.kdeplot(preds_0305_v2, shade=False)
plot.set(yticklabels=[], xticklabels = [])
sns.despine(left=True)

In [ ]:
initial = preds_0102

def scale_range (input, min, max):
    input = input.copy()
    input += -(np.min(input))
    input /= np.max(input) / (max - min)
    input += min
    return input

buckets = 10
raw_breakpoints = np.arange(0, buckets + 1) / buckets
breakpoints = scale_range(raw_breakpoints, np.min(initial), np.max(initial))
#breakpoints = raw_breakpoints
#breakpoints = np.stack([np.percentile(initial, b) for b in raw_breakpoints])

res = []
output_break_point = True
for new in (preds_0305, preds_0305_v2):
    initial_counts = np.histogram(initial, breakpoints)[0]
    new_counts = np.histogram(new, breakpoints)[0]

    df = pd.DataFrame({'Bucket': np.arange(1, buckets + 1), 'Breakpoint Value':breakpoints[1:], 'Initial Count':initial_counts, 'New Count':new_counts})
    df['Initial Percent'] = df['Initial Count'] / len(initial)
    df['New Percent'] = df['New Count'] / len(new)
    df['New Percent'][df['New Percent'] == 0] = 1e-4
    df['Initial Percent'][df['Initial Percent'] == 0] = 1e-4


    percents = df[['Initial Percent', 'New Percent', 'Bucket']] \
                 .melt(id_vars=['Bucket']) \
                 .rename(columns={'variable':'Population', 'value':'Percent'})


    df['PSI'] = (df['New Percent'] - df['Initial Percent']) * np.log(df['New Percent'] / (df['Initial Percent']))

    for idx, x in enumerate(df['PSI'].tolist()):
        if output_break_point:
            res = [[x] for x in df['Breakpoint Value'].tolist()]
            output_break_point = False
        res[idx].append(x)

last_bin = 0
for idx, l in enumerate(res):
    bin = l[0]
    print('[{:.3f}, {})'.format(last_bin, '{:.3f}'.format(bin) if idx != len(res) - 1 else 'inf'), end='')
    last_bin = bin
    for x in l[1:]:
        print('|{:.3%}'.format(x), end='')
    print()
arr = np.array(res)[:, 1:]
sums = arr.sum(axis=0)
print('Sum', ('|{:.3%}' * len(sums)).format(*sums))
